In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
from glob import glob
import os
print('done')

done


*Downloading session chats 1 and 2 from Github*

In [1]:
glob('*.txt')

NameError: name 'glob' is not defined

save the last session chat...

In [4]:
chats = 'chat_Aug15.txt'

In [5]:
# Let's merge both text files into one

# files_ = [f for f in glob('[jul4]*') if f.startswith('jul4')]
# files_

In [6]:
# # Python program to
# # demonstrate merging
# # of two files
  
# data = data2 = ""
  
# # Reading data from file1
# with open(files_[0], 'r', encoding='utf8') as fp:
#     data = fp.read()

# # Reading data from file2
# with open(files_[1], 'r', encoding='utf8') as fp:
#     data2 = fp.read()

# # Merging 2 files
# # To add the data of file2
# # from next line
# data += "\n"
# data += data2
  
# with open ('jul4merge.txt', 'w', encoding='utf8') as fp:
#     fp.write(data)

*Before June 12th Revokation...*

In [7]:
df = pd.read_csv('17may_69_students.csv')
df.shape

(69, 3)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69 entries, 0 to 68
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Email       69 non-null     object
 1   First Name  69 non-null     object
 2   Last Name   69 non-null     object
dtypes: object(3)
memory usage: 1.7+ KB


*After June 12th Revokation...*

In [9]:
df = pd.read_csv('session-4356-report-7_12_2022.csv').loc[:,['First Name', 'Last Name', 'Email']]
df.shape

(54, 3)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   First Name  54 non-null     object
 1   Last Name   54 non-null     object
 2   Email       54 non-null     object
dtypes: object(3)
memory usage: 1.4+ KB


In [11]:
for col in ['First Name', 'Last Name']:
    df[col] = df[col].apply(lambda x: x.lower().replace('_', ' ').replace('-', ' '))

In [12]:
df[['First Name', 'Last Name']].head(3)

,First Name,Last Name
0,kelvin,mwaka
1,etuonu,david
2,emmanuel,otiti


In [13]:
all_names = list(df['First Name']) + list(df['Last Name'])

*Read in the Connect Session Chats for the last session*

In [14]:
with open(chats, 'r') as fyle:
    lines = fyle.readlines()

*Making the read-out texts a dataframe to easily clean all rows...*

In [15]:
def make_df(texts, col='strings'):
    dd = pd.DataFrame(texts, columns=[col])
    return dd

In [16]:
dd = make_df(lines)
dd.head()

,strings
0,19:09:02 From Nndweleni Sundani to Everyone:\n
1,\tNndweleni Sundani\n
2,19:09:19 From Naomi Frank-Opigo to Everyone:\n
3,\tNaomi Ayibaemi Frank-Opigo\n
4,19:10:29 From Lawrence Krukrubo to Everyone:\n


Next, we clean the cols, convert each row to a list, explode the list and return a set of unique words

In [17]:
# First, we define a regex pattern for keeping only alphabets

import re
rgex = re.compile('[^a-zA-Z]')

In [18]:
def clean_lists_explode(df, col):
    # first clean all non-alphabetic characters
    df[col] = df[col].apply(lambda x: rgex.sub(' ', x).lower().strip())
    
    # Next split each row to a list
    df[col] = df[col].str.split(' ')
    
    # Next we explode the column of lists
    exploded = df.explode(col)[col]
    
    # next we keep a set of unique values
    exploded = set(exploded)
    
    print('Column Cleaned, Listed, Exploded!')
    
    # Finally, return a list of exploded values
    return list(exploded)

In [19]:
names_df = pd.DataFrame(all_names, columns=['names'])
names_df.head(3)

,names
0,kelvin
1,etuonu
2,emmanuel


*Now, we explode the chats dataframe..*

In [20]:
exploded1 = clean_lists_explode(dd, 'strings')

Column Cleaned, Listed, Exploded!


In [21]:
len(exploded1)

46

*Now, we explode the all names dataframe..*

In [22]:
exploded2 = clean_lists_explode(names_df, 'names')

Column Cleaned, Listed, Exploded!


In [23]:
len(exploded2)

111

Let's now keep only common words in both exploded and all_names...

In [24]:
commons = list(set(exploded1).intersection(set(exploded2)))
print(f'len commons: {len(commons)}')

len commons: 11


In [25]:
def find_present(commons, all_names=all_names, df=df):
    ddf = pd.DataFrame([], columns=['present'])
    
    for name in commons:
        long_name = [i for i in all_names if name in i][0]
        if long_name:
            try:
                q = f'`First Name` == "{long_name}"'
                tmp = df.query(q)[['First Name', 'Last Name']].values
                assert len(tmp) > 0
            except AssertionError:
                q = f'`Last Name` == "{long_name}"'
                tmp = df.query(q)[['First Name', 'Last Name']].values
        
            name_ = ['_'.join(i) for i in tmp][0]
            ddf = ddf.append({'present':name_}, ignore_index=True)
            
    # drop Duplicates
    ddf.drop_duplicates(inplace=True)
    
    # sort names alphabetically
    ddf.sort_values(by='present', inplace=True)
    
    # reset_index
    ddf.index = range(len(ddf))
    
    # replace empty space with underscore
    ddf.present = ddf.present.apply(lambda x: x.replace(' ','_').strip())
    
    return ddf

In [26]:
present = find_present(commons)

In [27]:
print(f'total present: {len(present)}')

total present: 5


In [28]:
present

,present
0,abdullah_ajao
1,abdulmalik_ismail
2,benjamin_onyenekwe
3,naomi_frank_opigo
4,nndweleni_sundani
